# Preparation

## Database Connection

In [3]:
# koneksi dari Python ke PostgreSQL
import psycopg2

# bulk INSERT dari Python ke PostgreSQL.
from psycopg2.extras import execute_values

In [69]:
# Koneksi ke database PostgreSQL
conn = psycopg2.connect(
    database='Tugas',
    user='postgres',
    password='biya2804',
    host='localhost',
    port='5432',
)

cur = conn.cursor()

# Downloading Webpages

In [6]:
# untuk membuat nama file unik berdasarkan URL halaman web.
import hashlib

# Regex
import re

# Untuk mendapatkan tanggal & waktu saat mengakses halaman web.
import time

# Untuk melakukan operasi request HTTP.
from urllib.request import Request, urlopen

## Robots.txt

In [8]:
# We're going to scrap information about property prices in Bali and Bandung.
# We have set our eyes on traveloka.com, to be precise:
#
# https://www.traveloka.com/en-id/accommodation/villa/indonesia
#
# Is this URL allowed to be accessed by our scrapper robot? The answer is
# in https://www.traveloka.com/robots.txt

## Webpage Metadata

Saya akan melakukan scrapping menggunakan 4 url, 2 URL untuk Villa (di Bali dan Bandung) dan 2 URL untuk Apartment (di Bali dan Bandung).<br>
Data yang akan diambil:<br>
1. Nama tempat penginapan --> identifikasi unik tempat penginapan
2. Harga --> analisis tren harga berdasarkan lokasi
3. Rating --> kualitas penginapan menururt pelanggan
4. Lokasi 

In [11]:
PROPERTY_TYPE_APARTMENT = 'apartment'
PROPERTY_TYPE_VILLA = 'villa'

In [12]:
# we hardcode only 4 URLs
urls = [
    'https://www.traveloka.com/en-id/accommodation/villa/indonesia/region/bali-102746',
    'https://www.traveloka.com/en-id/accommodation/villa/indonesia/city/bandung-103859',
    'https://www.traveloka.com/en-id/accommodation/apartment/indonesia/region/bali-102746',
    'https://www.traveloka.com/en-id/accommodation/apartment/indonesia/city/bandung-103859',
]

# to extract the domain name, i.e. www.traveloka.com
pattern_source = re.compile(r'//(.+?)/')

# To extract the location name appended in the URL, e.g. 
# bali, bandung
pattern_location = re.compile(r'/([^/]+)-\d+')

# To extract the property type given in the URL's paramater, e.g. 
# apartment, villa
pattern_property_type = re.compile(r'/accommodation/([^/]+)/')

webpages = []

# looping melalui urls
for i in range(len(urls)):
    url = urls[i]
    source = pattern_source.search(url).group(1)
    location = pattern_location.search(url).group(1)
    property_type = pattern_property_type.search(url).group(1)
    if property_type == 'apartment':
        property_type = PROPERTY_TYPE_APARTMENT
    elif property_type == 'villa':
        property_type = PROPERTY_TYPE_VILLA
    else:
        property_type = None

    # Generate a filename to store the downloaded HTML page.
    filepath = 'data/' + hashlib.md5(url.encode()).hexdigest() + '.html'

    webpage = [source, location, property_type, url, filepath, '']
    webpages.append(webpage)

In [13]:
webpages

[['www.traveloka.com',
  'bali',
  'villa',
  'https://www.traveloka.com/en-id/accommodation/villa/indonesia/region/bali-102746',
  'data/7b97c935193cd93b166803a5e401151f.html',
  ''],
 ['www.traveloka.com',
  'bandung',
  'villa',
  'https://www.traveloka.com/en-id/accommodation/villa/indonesia/city/bandung-103859',
  'data/4f8128da8af39d6bc418aa6c3672a9e9.html',
  ''],
 ['www.traveloka.com',
  'bali',
  'apartment',
  'https://www.traveloka.com/en-id/accommodation/apartment/indonesia/region/bali-102746',
  'data/e0e6571500ab1bc2da6c1f326fb70640.html',
  ''],
 ['www.traveloka.com',
  'bandung',
  'apartment',
  'https://www.traveloka.com/en-id/accommodation/apartment/indonesia/city/bandung-103859',
  'data/970fb87a2c8dbe040a10c9942d50cbd5.html',
  '']]

## HTTP Request

dari 4 url yang disimpan pada variabel urls, banyak sekali page/halaman dari tiap-tiap url nya namun pada scrapping kali ini saya hanya akan mengambil 4 page dari tiap url, sehingga nanti akan dilakukan looping.

In [16]:
import requests # Untuk mengambil halaman web melalui HTTP request
import time     # Untuk mengelola waktu, misalnya penundaan antar permintaan
import random   # Untuk menghasilkan jeda acak agar tidak terdeteksi sebagai bot
from bs4 import BeautifulSoup # Untuk parsing HTML dan mengekstrak informasi dari halaman web

urls = [
    'https://www.traveloka.com/en-id/accommodation/villa/indonesia/region/bali-102746',
    'https://www.traveloka.com/en-id/accommodation/villa/indonesia/city/bandung-103859',
    'https://www.traveloka.com/en-id/accommodation/apartment/indonesia/region/bali-102746',
    'https://www.traveloka.com/en-id/accommodation/apartment/indonesia/city/bandung-103859',
]

# Menyimpan metadata halaman web
webpages = []

# Headers untuk request HTTP
headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.traveloka.com/en-id",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8"
}

# Download page
for url in urls:
    location = pattern_location.search(url).group(1)
    property_type = pattern_property_type.search(url).group(1)
    if property_type == 'apartment':
        property_type = PROPERTY_TYPE_APARTMENT
    elif property_type == 'villa':
        property_type = PROPERTY_TYPE_VILLA
    else:
        property_type = None
    for page in range(1, 5):  # Mengunduh 4 halaman per URL
        full_url = f"{url}?page={page}"
        filepath = f"data/{hashlib.md5(full_url.encode()).hexdigest()}.html"

        try:
            response = requests.get(full_url, headers=headers, timeout=10)
            response.raise_for_status()

            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(response.text)

            print(f"✅ Berhasil mengunduh {full_url}")
            accessed_at = time.strftime('%Y-%m-%d %H:%M:%S %z')

            webpages.append((source, location, property_type, url, filepath, accessed_at))
            
        except requests.exceptions.RequestException as e:
            print(f"Gagal mengunduh {full_url}: {e}")

        time.sleep(random.uniform(1, 3))  # Jeda untuk menghindari deteksi bot

print("✅ Semua halaman telah diunduh.")

✅ Berhasil mengunduh https://www.traveloka.com/en-id/accommodation/villa/indonesia/region/bali-102746?page=1
✅ Berhasil mengunduh https://www.traveloka.com/en-id/accommodation/villa/indonesia/region/bali-102746?page=2
✅ Berhasil mengunduh https://www.traveloka.com/en-id/accommodation/villa/indonesia/region/bali-102746?page=3
✅ Berhasil mengunduh https://www.traveloka.com/en-id/accommodation/villa/indonesia/region/bali-102746?page=4
✅ Berhasil mengunduh https://www.traveloka.com/en-id/accommodation/villa/indonesia/city/bandung-103859?page=1
✅ Berhasil mengunduh https://www.traveloka.com/en-id/accommodation/villa/indonesia/city/bandung-103859?page=2
✅ Berhasil mengunduh https://www.traveloka.com/en-id/accommodation/villa/indonesia/city/bandung-103859?page=3
✅ Berhasil mengunduh https://www.traveloka.com/en-id/accommodation/villa/indonesia/city/bandung-103859?page=4
✅ Berhasil mengunduh https://www.traveloka.com/en-id/accommodation/apartment/indonesia/region/bali-102746?page=1
✅ Berhasil 

In [17]:
webpages

[('www.traveloka.com',
  'bali',
  'villa',
  'https://www.traveloka.com/en-id/accommodation/villa/indonesia/region/bali-102746',
  'data/8d5273324d9845ed2e53a94237dad9fb.html',
  '2025-04-19 06:13:14 +0700'),
 ('www.traveloka.com',
  'bali',
  'villa',
  'https://www.traveloka.com/en-id/accommodation/villa/indonesia/region/bali-102746',
  'data/57394a6df840890442df137b22468770.html',
  '2025-04-19 06:13:19 +0700'),
 ('www.traveloka.com',
  'bali',
  'villa',
  'https://www.traveloka.com/en-id/accommodation/villa/indonesia/region/bali-102746',
  'data/4bfdf428a295d3747c3cf5a0fd97b4c7.html',
  '2025-04-19 06:13:24 +0700'),
 ('www.traveloka.com',
  'bali',
  'villa',
  'https://www.traveloka.com/en-id/accommodation/villa/indonesia/region/bali-102746',
  'data/8fc3da55a9485dadfb2dee9024748653.html',
  '2025-04-19 06:13:32 +0700'),
 ('www.traveloka.com',
  'bandung',
  'villa',
  'https://www.traveloka.com/en-id/accommodation/villa/indonesia/city/bandung-103859',
  'data/3b70d99591cc34c7a0

## Database

In [19]:
# membuat tipe data ENUM di PostgreSQL dengan nama property_types
cur.execute(
    'CREATE TYPE property_types AS ENUM (%s, %s);',
    (PROPERTY_TYPE_APARTMENT, PROPERTY_TYPE_VILLA)
)

In [20]:
# untuk membuat tabel webpages di PostgreSQL 
# yang akan menyimpan metadata tentang halaman web.
cur.execute('''
    CREATE TABLE webpages (
        id serial PRIMARY KEY,
        source varchar(32),
        location varchar(32),
        property_type property_types,
        url varchar,
        filepath varchar,
        accessed_at timestamptz
    );
''')

In [21]:
# untuk melakukan bulk insert ke tabel webpages di PostgreSQL
execute_values(
    cur,
    '''INSERT INTO webpages (
           source,
           location,
           property_type,
           url,
           filepath,
           accessed_at
       ) 
       VALUES %s
    ''',
    webpages,
)

In [22]:
# untuk menyimpan perubahan ke database
conn.commit()

# Information Extraction

## Webpage Metadata (from DB)

In [25]:
# mengambil hanya lokasi unik
cur.execute('SELECT DISTINCT location FROM webpages;')
cur.fetchall()

[('bandung',), ('bali',)]

In [26]:
# mengambil metadata halaman web dari tabel webpages di PostgreSQL 
# berdasarkan lokasi dan tipe properti
cur.execute('''
    SELECT
        id,
        source,
        filepath,
        property_type
    FROM webpages
    WHERE
        location in (%s, %s)
        AND property_type in (%s, %s);
    ''',
    ('bali', 'bandung', PROPERTY_TYPE_VILLA, PROPERTY_TYPE_APARTMENT),
)
webpages = cur.fetchall()

In [27]:
webpages

[(1,
  'www.traveloka.com',
  'data/8d5273324d9845ed2e53a94237dad9fb.html',
  'villa'),
 (2,
  'www.traveloka.com',
  'data/57394a6df840890442df137b22468770.html',
  'villa'),
 (3,
  'www.traveloka.com',
  'data/4bfdf428a295d3747c3cf5a0fd97b4c7.html',
  'villa'),
 (4,
  'www.traveloka.com',
  'data/8fc3da55a9485dadfb2dee9024748653.html',
  'villa'),
 (5,
  'www.traveloka.com',
  'data/3b70d99591cc34c7a0e21cbed84aa92f.html',
  'villa'),
 (6,
  'www.traveloka.com',
  'data/40527909bdc897e3aa1a2e22d13afab7.html',
  'villa'),
 (7,
  'www.traveloka.com',
  'data/5f7c546a516722ec9672adfcf5f8b21f.html',
  'villa'),
 (8,
  'www.traveloka.com',
  'data/d4472684482a73574482b72217b328fc.html',
  'villa'),
 (9,
  'www.traveloka.com',
  'data/2553846eadabce73ab9c8685b302ab25.html',
  'apartment'),
 (10,
  'www.traveloka.com',
  'data/aa71b8a8bc8c59b326cbb9927df78259.html',
  'apartment'),
 (11,
  'www.traveloka.com',
  'data/cab9859be1fe57d18a4a8328219b2d1b.html',
  'apartment'),
 (12,
  'www.trave

## Scraping

In [51]:
# regex
# To get the district name (kecamatan).
pattern_district = re.compile(r'^[^,]+')

# To get the city/regency name (kabupaten/kota).
pattern_city_regency = re.compile(r'(?<=, )\w+')

# To get the property price (preserve only the digits).
pattern_price = re.compile(r'\D')

properties = []
for i in range(len(webpages)):
    print('Scraping webpage {0} of {1}'.format(i+1, len(webpages)))
    
    webpage_id = webpages[i][0]
    source = webpages[i][1]
    filepath = webpages[i][2]
    property_type = webpages[i][3]

    with open(filepath, 'r', encoding='utf-8') as f:
        # Initialize the beautiful soup.
        soup = BeautifulSoup(f, 'lxml')

        # Get all HTML elements that contain our precious information.
        elements = soup.find_all(
            'div',
            {
                'class': 'css-1dbjc4n r-14lw9ot r-awg2lu r-1dzdj1l r-rs99b7 r-1loqt21 r-18u37iz r-1udh08x r-1otgn73 r-1i6wzkk r-lrvibr',
            },
        )

        # Iterate through all elements to extract information of each 
        # property. Ideally speaking, based on our observation on the 
        # structure of the HTML, each HTML contains exactly one property.
        for e in elements:
            # E.g. 'The Catur Villa Seminyak'.
            title = e.find('div', {'class': 'css-1dbjc4n r-13awgt0 r-18u37iz'}).get_text(strip=True)

            # The location part contains both district and city/regency 
            # E.g. 'Anyer, Banten' --> Anyer for district, Banten for city_regency
            location = e.find('div', {'class': 'css-1dbjc4n r-1awozwy r-1kbdv8c r-18u37iz r-dnmrzs'}).get_text(strip=True)
            district = pattern_district.search(location).group(0) if pattern_district.search(location) else "Tidak ditemukan"
            city_regency = pattern_city_regency.search(location).group(0) if pattern_city_regency.search(location) else "Tidak ditemukan"

            # Strip out non-digit from the price information. E.g.
            # Rp 1.368.260 -> 1368260
            price = e.find('div', {'class': 'css-901oao r-mtym6t r-adyw6z r-1w9mtv9 r-1kfrs79 r-135wba7 r-1ff274t'}).contents[0]
            price = pattern_price.sub('',price.get_text(strip=True)) if pattern_price else "0"

            # rating
            # Cek apakah rating bisa ditemukan dengan aman
            rating_elem = e.find('div', attrs={'data-testid': 'popular-hotel-rating'})
            rating = rating_elem.get_text(strip=True) if rating_elem else None
             # Pastikan rating berupa angka, jika tidak, ubah menjadi None
            rating = float(rating) if rating and rating.replace('.', '', 1).isdigit() else None
        
            # More precious information might be available in the 
            # property detail URL (i.e. when users click on the property 
            # listing). We collected more URLs to scrap later. Collecting 
            # valid URLs recursively like this is called 'crawling'.
            detail_url = source + e.find('a').get('href')

            properties.append((
                webpage_id,
                title,
                property_type,
                district,
                city_regency,
                price,
                detail_url,
                rating
            ))

        print('Scraped {0} properties'.format(len(elements)))

Scraping webpage 1 of 16
Scraped 20 properties
Scraping webpage 2 of 16
Scraped 20 properties
Scraping webpage 3 of 16
Scraped 20 properties
Scraping webpage 4 of 16
Scraped 20 properties
Scraping webpage 5 of 16
Scraped 20 properties
Scraping webpage 6 of 16
Scraped 20 properties
Scraping webpage 7 of 16
Scraped 20 properties
Scraping webpage 8 of 16
Scraped 20 properties
Scraping webpage 9 of 16
Scraped 20 properties
Scraping webpage 10 of 16
Scraped 20 properties
Scraping webpage 11 of 16
Scraped 20 properties
Scraping webpage 12 of 16
Scraped 20 properties
Scraping webpage 13 of 16
Scraped 20 properties
Scraping webpage 14 of 16
Scraped 20 properties
Scraping webpage 15 of 16
Scraped 20 properties
Scraping webpage 16 of 16
Scraped 20 properties


## Database

In [57]:
# Buat tabel di PostgreSQL untuk menyimpan informasi yang sudah di scrapped.
cur.execute('''
    CREATE TABLE properties (
    id serial PRIMARY KEY,
    webpage_id integer REFERENCES webpages(id),  -- Pastikan tabel webpages punya kolom id
    title varchar(255),
    property_type varchar(50), 
    district varchar(50),
    city_regency varchar(50),
    price bigint,
    detail_url varchar(255), 
    rating DOUBLE PRECISION
);
''')

In [59]:
# memasukan data scraped ke postgresql.
execute_values(
    cur,
    '''INSERT INTO properties (
           webpage_id,
           title,
           property_type,
           district,
           city_regency,
           price,
           detail_url,
           rating
       ) 
       VALUES %s
    ''',
    properties,
)

In [61]:
conn.commit()

# Quick Analysis

In [63]:
# Bagaimana distribusi harga villa di bali
cur.execute('''
    SELECT
        MIN(price) AS min_price,
        MAX(price) AS max_price,
        AVG(price) AS avg_price
    FROM properties
    WHERE webpage_id = 1;
''')
cur.fetchall()

[(382407, 5077791, Decimal('1538715.800000000000'))]

Dari data yang didapat untuk harga permalam villa di bali:<br>
Harga termurah villa di bali yaitu Rp 382.407.<br>
Harga termahal villa di bali yaitu Rp 5.077.791.<br>
Rata-rata harga villa di bali yaitu Rp 1.538.715,8.<br>

In [71]:
import pandas as pd
# Menampilkan akomodasi dengan rating tertinggi (10 tertinggi)
query = """
SELECT distinct title, district, city_regency, rating, price, property_type
FROM properties
WHERE rating IS NOT NULL
ORDER BY rating DESC
LIMIT 10;
"""
df = pd.read_sql(query, conn)
print(df)

                                         title        district city_regency  \
0                           Dusun Bedugul Asri         Bedugul      Tabanan   
1  Savya ApartHotel by Kozystay - Bandung City     Asia Afrika      Bandung   
2               Villa Semesta Resor Dago Pakar       Dago Atas         Dago   
3                    The Dewi Eco Bamboo Villa       Kintamani       Bangli   
4                         Villa Kembar Lembang         Lembang      Bandung   
5         youre at - Grand Setiabudi Apartment       Setiabudi      Bandung   
6                                  K Club Ubud     Tegallalang         Ubud   
7          PARAHYANGAN RESIDENCES by AYA Stays     Ciumbuleuit      Bandung   
8                  Citadines Berawa Beach Bali          Canggu         Kuta   
9                              Bestah Coliving  South Denpasar     Denpasar   

   rating    price property_type  
0     9.6  1024598         villa  
1     9.5  1528451     apartment  
2     9.4   762091       

C:\Users\ASUS\AppData\Local\Temp\ipykernel_13432\466066442.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Data ini bisa digunakan untuk melihat penginapan dengan rating terbaik, cocok untuk digunakan orang-orang kalangan atas yang mungkin mempertimbangkan dari segi kenyamanan 

In [75]:
# Mencari Akomodasi dengan Harga di Bawah Rata-rata
query = """
SELECT distinct title, district, city_regency, rating, price
FROM properties
WHERE price < (SELECT AVG(price) FROM properties)
ORDER BY price ASC;
"""
df = pd.read_sql(query, conn)
print(df)

                                                title        district  \
0                                    Adelia Residence   West Denpasar   
1               Azhimah Rooms At Jatinangor Near IPDN      Jatinangor   
2   The Suites Metro Apartment Bandung by Zaenal King     Batununggal   
3         The Suite Metro Apartemen ( King Property )     Batununggal   
4         Apartment The Suites Metro by Rusdi Kingpro     Batununggal   
5                    Gateway Pasteur Apartemen By SPH         Cicendo   
6                                      Villa de Rossa         Lembang   
7                                 Bantal Guling Villa         Lembang   
8                      The Jarrdin Apartment by Omami      Cihampelas   
9                        Easton Park Apartment By 7RM      Jatinangor   
10                         High Livin Apartment Baros          Cimahi   
11        Apartement Grand Asia Afrika By Damar Akbar     Asia Afrika   
12                               Deva Bali Aparteme

C:\Users\ASUS\AppData\Local\Temp\ipykernel_13432\1326122290.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Menampilkan akomodasi yang lebih murah dari rata-rata semua properti baik di bali maupun di bandung, dari 320 data terdapat 52 villa dan apartment yang harganya dibawah rata-rata.<br>
Data ini bisa digunakan apabila budget user/orang yang liburan pas-pas an atau mencari penginapan dengan harga terjangkau.

# Cleanup

## Database

In [67]:
# Close communication with the database
cur.close()
conn.close()